In [2]:
from maze import Maze
import random

In [43]:
maze_height = 100
maze_width = 100
maze = Maze(maze_height, maze_width, 1, 1)
maze.generate_maze()
maze.show_maze(show_path = False)

Obstacles count:  9786
Obstacles count:  9616
|S|| ||#|| ||#|| ||#||#||#|| ||#|| ||#|| ||#||#|| ||#||#||#|| || || ||#|| ||#||#||#|| ||#||#||#|| ||#||#||#||#||#||#|| ||#|| || || ||#||#|| ||#||#|| || ||#||#||#||#||#||#|| || || ||#|| || || ||#||#||#||#||#|| ||#|| ||#|| ||#|| || || ||#|| ||#||#|| || ||#||#||#||#||#|| ||#||#|| ||#||#|| || || ||#||#|
| || ||#||#||#||#|| || || || ||#||#||#||#|| || ||#|| ||#||#||#||#||#||#||#|| || ||#|| ||#|| ||#|| ||#||#||#|| ||#||#||#|| ||#||#|| ||#||#||#|| || ||#||#||#||#||#||#|| || ||#||#|| || ||#||#||#||#||#|| ||#|| || ||#|| ||#||#||#||#||#|| ||#||#|| || ||#||#||#|| ||#|| ||#||#||#|| ||#||#|| ||#|| || ||#|| |
| || || ||#||#|| ||#||#||#||#||#||#|| ||#||#|| || ||#|| || ||#|| || ||#||#||#|| ||#|| ||#||#||#|| || ||#|| ||#||#||#|| ||#||#||#||#||#||#||#|| || ||#|| ||#||#||#||#||#|| ||#||#||#|| ||#|| || || ||#||#||#|| ||#|| ||#||#|| || ||#||#|| ||#||#||#||#||#||#||#||#|| ||#|| || ||#|| || ||#|| ||#|| || || ||#|
| || || || ||#||#|| || || ||#||#||#|| || ||#|| ||#|

In [44]:
n_population = 2
mutation_rate = 0.1
n_generations = 100
n_parents = n_population//2
steps = maze_height*maze_width-1
solution_path = []
population = generate_init_population(n_population, steps)

for generation in range(n_generations):
    print('Generation: ', generation)
    fitness = get_fitness(population, maze)
    print('Fitness: ', fitness)
    if min(fitness) == 0:
        solution_path = population[fitness.index(0)]
        break
    parents = select_parents(population, fitness, n_parents)
    children = get_next_generation(population, fitness, parents)
    population = parents + children
    # print('New Population: ', population)
if solution_path == []:
    print('Solution not found')
print('Solution: ', solution_path)



Generation:  0
Fitness:  [1783933, 1558080]
Generation:  1
Fitness:  [1558080, 1523827]
Generation:  2
Fitness:  [1523827, 1611823]
Generation:  3
Fitness:  [1523827, 1408751]
Generation:  4
Fitness:  [1408751, 1462519]
Generation:  5
Fitness:  [1408751, 1543243]
Generation:  6
Fitness:  [1408751, 1456219]
Generation:  7
Fitness:  [1408751, 1547148]
Generation:  8
Fitness:  [1408751, 1439887]
Generation:  9
Fitness:  [1408751, 1524025]
Generation:  10
Fitness:  [1408751, 1628696]
Generation:  11
Fitness:  [1408751, 1448002]
Generation:  12
Fitness:  [1408751, 1443755]
Generation:  13
Fitness:  [1408751, 1460632]
Generation:  14
Fitness:  [1408751, 1575171]
Generation:  15
Fitness:  [1408751, 1313258]
Generation:  16
Fitness:  [1313258, 1384245]
Generation:  17
Fitness:  [1313258, 1470168]
Generation:  18
Fitness:  [1313258, 1528074]
Generation:  19
Fitness:  [1313258, 1504504]
Generation:  20
Fitness:  [1313258, 1419516]
Generation:  21
Fitness:  [1313258, 1517859]
Generation:  22
Fitn

In [5]:
def generate_init_population(n_population, steps):
    population = []
    for i in range(n_population):
        route = []
        for _ in range(steps):
            route.append(random.randint(0, 1))
        population.append(route)
    return population

In [6]:
def get_fitness(population, maze):
    fitness = []
    for route in population:
        fitness.append(get_route_fitness(route, maze))
    return fitness

In [13]:
def get_route_fitness(route,maze):
    start = 0
    total_fitness = 0
    end = maze.height*maze.width-1
    for step in route:
        if step == 0:
            start += maze.width
        else:
            start += 1
        if start == maze.height*maze.width-1:
            total_fitness = 0
            return total_fitness
        if start in maze.obstacle_loc:
            total_fitness += end-start
    rev_route = route[::-1]
    for step in rev_route:
        if step == 0:
            end -= maze.width
        else:
            end -= 1
        if end in maze.obstacle_loc:
            total_fitness += end
    return total_fitness

In [8]:
def get_next_generation(population, fitness, parents):
    next_generation = []
    for i in range(len(population)//2):
        child = crossover(parents)
        child = mutate(child)
        next_generation.append(child)
    return next_generation

In [23]:
def select_parents(population, fitness, n_parents):
    
    parents = []
    for i in range(n_parents):
        min_fitness = min(fitness)
        min_fitness_index = fitness.index(min_fitness)
        parents.append(population[min_fitness_index])
        fitness[min_fitness_index] = max(fitness)+1
        
    return parents

In [16]:
def crossover(parents):
    child = []
    crossover_point = random.randint(0, len(parents[0]))
    parent_1 = parents[random.randint(0, len(parents)-1)]
    parent_2 = parents[random.randint(0, len(parents)-1)]
    child.extend(parent_1[:crossover_point])
    child.extend(parent_2[crossover_point:])
    return child

In [11]:
def mutate(child):
    for i in range(len(child)):
        if random.random() < mutation_rate:
            if child[i] == 0:
                child[i] = 1
            else:
                child[i] = 0
    return child